# 4 - Feras Formidáveis

## 4.2  Stop right now, thank you very much

### Introdução:

O objetivo desta atividade é implementar uma estratégia de Parada Antecipada (Early Stopping) no
processo de treino da rede neural feita em PyTorch. Utilizou-se dados simples para verificar o funcionamento, empregando X camadas com uma função de ativação linear. Boa parte do algoritmo foi adaptada dos materiais de aula do Daniel Cassar [1,2].



A Parada Antecipada é um método para estimar um número de épocas adequado no treinamento do algoritmo, evitando que ele sofra subajuste ou sobre-ajuste. Isso significa que, no final de cada época, deve-se calcular a precisão da classificação nos dados de validação e terminar o treinamento quando a precisão parar de melhorar. [3] Ou seja, em problemas de regressão, é necessário criar uma curva de aprendizado, verificando a progressão da função perda para os dados de treino e de validação e definindo qual a melhor época para parar o modelo.

### Desenvolvimento:

Importando as bibliotecas necessárias [4-7]:

In [186]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns

Definindo a rede neural, baseada em uma Multi Layer Perceptron (MLP), em que aplica-se transformações lineares nos dados das camadas com base em uma função de ativação

In [187]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, camadas_ocultas, funcao_ativacao, num_targets
    ):
        # o método super recupera todas as funções anteriores (no caso da classe Module)
        super().__init__()

        arquitetura = []

        # primeira camada oculta
        arquitetura.append(nn.Linear(num_dados_entrada, camadas_ocultas[0]))
        arquitetura.append(funcao_ativacao)

        # demais camadas ocultas
        for i in range(1, len(camadas_ocultas)):
            arquitetura.append(nn.Linear(camadas_ocultas[i-1], camadas_ocultas[i]))
            arquitetura.append(funcao_ativacao)

        # camada de saída
        arquitetura.append(nn.Linear(camadas_ocultas[-1], num_targets))

        self.camadas = nn.Sequential(*arquitetura)

    # similar ao método __call__    
    def forward(self, x):
        x = self.camadas(x)
        return x

Instanciando a MLP, contendo 4 dados de entrada, 2 camadas ocultas, função de ativação sigmoidal e 1 dado como output

In [188]:
NUM_DADOS_DE_ENTRADA = 4
CAMADAS_OCULTAS = [2, 3]
FUNCAO_ATIVACAO = nn.Sigmoid()
NUM_DADOS_DE_SAIDA = 1  

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, CAMADAS_OCULTAS, FUNCAO_ATIVACAO, NUM_DADOS_DE_SAIDA
)

Definindo o otimizador do problema, o qual realiza as etapas de backpropagation, com uma taxa de aprendizado em 0.01 e o Erro Quadrático Médio (MSE) como função de perda

In [189]:
# learning rate (ln)
TAXA_DE_APRENDIZADO = 0.01 

# modo de atualizar os parâmetros, usou-se o estocástico gradient descent (SGD)
otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)

# função a ser minimizada, usou-se a do erro quadrático médio (MSE)
fn_perda = nn.MSELoss()

Criando exemplos simples para testar o modelo, convertendo-os em tensores

In [190]:
x_treino = [
  [1.0, 0.0, -1.0, 2.3],
  [3.0, -1.0, 0.5, 9.1],
  [0.7, 8.6, 3.0, 2.6],
  [1.0, 1.0, -1.0, 4.4],
]
y_treino = [1, 0, 0.2, 0.5]

x_val = [
  [6.0, 1.0, -6.0, 2.9],
  [4.0, -2.5, 0.2, 7.8],
  [-1.7, 5.6, 2.2, -3.4],
  [8.0, 1.0, 1.0, 5.2],
]
y_val = [2, -1.3, 0.0, 3.5]

x_treino = torch.tensor(x_treino)
# Assim como o reshape pro numpy, converte em uma coluna (1), usando o número de linhas necessárias (-1)
y_treino = torch.tensor(y_treino).view(-1,1)

x_val = torch.tensor(x_val)
y_val = torch.tensor(y_val).view(-1,1)


Treinando o modelo para os dados de treino e teste, definindo inicialmente um número de épocas como 1000, o que é exagerado para ilustrar o funcionamento da Parada Antecipada. O código foi adaptado de fontes online [8,9], definindo uma paciência de 1 e delta de 0.001

In [191]:
# Período de treinamento inicial
NUM_EPOCAS = 1000
epocas = [i for i in range(NUM_EPOCAS)]
perdas_treino = []
perdas_val = []

# Parâmetros do early stopping
PACIENCIA = 1
DELTA = 0.001
i = 0
min_perda_val = float('inf')

for epoca in range(NUM_EPOCAS):
    # Treinamento
    minha_mlp.train()
    yp_treino = minha_mlp(x_treino) # forward pass
    otimizador.zero_grad() # zero grad
    loss = fn_perda(y_treino, yp_treino) # função de perda
    perdas_treino.append(loss)
    loss.backward() # backpropagation
    otimizador.step() # atualiza parâmetros - passo

    # Validação
    minha_mlp.eval()
    with torch.no_grad():  # não calcula os gradientes locais
        yp_val = minha_mlp(x_val)
        val_loss = fn_perda(y_val, yp_val)
        perdas_val.append(val_loss) 

    # Parada Antecipada
    if val_loss < min_perda_val:
        min_perda_val = val_loss
        i = 0
        
    elif val_loss > (min_perda_val + DELTA):
        i += 1
        if i >= PACIENCIA:
            print(f'Parada Antecipada na época {epoca}')
            break

for treino, val, epoca in zip(perdas_treino, perdas_val, epocas):
    print(f'Época {epoca}: Perda de treino {treino:4f}, perda de validação {val:4f}')

Parada Antecipada na época 292
Época 0: Perda de treino 0.428693, perda de validação 4.711963
Época 1: Perda de treino 0.408222, perda de validação 4.669833
Época 2: Perda de treino 0.389189, perda de validação 4.629847
Época 3: Perda de treino 0.371492, perda de validação 4.591882
Época 4: Perda de treino 0.355040, perda de validação 4.555826
Época 5: Perda de treino 0.339746, perda de validação 4.521574
Época 6: Perda de treino 0.325528, perda de validação 4.489027
Época 7: Perda de treino 0.312313, perda de validação 4.458090
Época 8: Perda de treino 0.300030, perda de validação 4.428677
Época 9: Perda de treino 0.288614, perda de validação 4.400704
Época 10: Perda de treino 0.278005, perda de validação 4.374094
Época 11: Perda de treino 0.268146, perda de validação 4.348774
Época 12: Perda de treino 0.258984, perda de validação 4.324674
Época 13: Perda de treino 0.250472, perda de validação 4.301730
Época 14: Perda de treino 0.242563, perda de validação 4.279881
Época 15: Perda de 

Perceba que a estratégia funcionou, forçando a parada do treinamento quando a perda da validação começa a aumentar

Criando dados de teste

In [192]:
x_teste = [
  [2.1, 4.8, -0.4, -1.0],
  [2.0, -1.9, 3.1, 7.3],
  [0.0, 1.9, 8.6, 6.4],
]

y_teste = [1.3, 0.3, 8.4]

x_teste = torch.tensor(x_teste)
y_teste = torch.tensor(y_teste).view(-1,1)

Treinando a rede neural para os dados de teste

In [194]:
# evaluate, avisa que a mlp está na etapa de previsão e mostra informações
minha_mlp.eval()

# dentro do with indica que não usará os gradientes locais (no_grad()), pois já otimizou e gasta recurso
with torch.no_grad():
    y_pred = minha_mlp(x_teste)

y_pred

tensor([[0.4071],
        [0.4309],
        [0.4044]])

Novamente, as previsões são sub-ótimas, pois o foco está em implementar o método de Early Stopping

### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 5.2 - Notebook PyTorch.ipynb". Material de Aula, 2025.

[2] CASSAR, Daniel. "ATP-303 NN 6.2 - Construindo e treinando uma rede neural com Lightning.ipynb". Material de Aula, 2025.

[3] Material sobre Early stopping. https://www.deeplearningbook.com.br/usando-early-stopping-para-definir-o-numero-de-epocas-de-treinamento/

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c